## Osadzenia wektorowe (vector embeddings)

Imię i nazwisko:

Celem laboratorium jest:
* Zapoznanie z się z koncepcją zamiany tekstu na wektory
* Implementacja algorytmu CBOW

Punktacja:
* działający kod 6 pkt.
* wnioski 2 pkt.

Mikolov, Tomas, et al. "Efficient estimation of word representations in vector space." arXiv preprint arXiv:1301.3781 (2013).
<img src="https://github.com/asztyber/jak_dziala_gpt_lab/blob/main/pic/cbow.png?raw=true" alt="Algorytm CBOW" width="500" height="300">

#### Dane uczące
Zbiór danych WikiText2 (wybrane artykuły z Wikipedii)
* Więcej tu: https://blog.salesforceairesearch.com/the-wikitext-long-term-dependency-language-modeling-dataset/

In [ ]:
!pip install datasets

In [ ]:
import torch
from datasets import load_dataset
import nltk # podstawowa biblioteka do przetwarzania języka naturalnego
import re
import numpy as np
from collections import Counter
import random
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
nltk.download('punkt_tab')

### Przygotowanie danych uczących

In [ ]:
ds = load_dataset("Salesforce/wikitext", "wikitext-2-raw-v1", split='train')

In [ ]:
ds

In [ ]:
# liczba linii tekstu w danych
len(ds['text'])

In [ ]:
text = ds['text']

In [ ]:
text[:10]

In [ ]:
# wszystko poza literami i cyframi zamieniamy na spacje za pomocą wyrażeń regularnych
text = [re.sub(r'[^a-zA-Z0-9]', ' ', line) for line in text]

In [ ]:
text[:10]

In [ ]:
# wszystkie litery zamieniamy na małe
text = [line.lower() for line in text]

In [ ]:
text[:10]

In [ ]:
# każdą linię tekstu dzielimy na słowa (tokenizacja)
text = [nltk.word_tokenize(line) for line in text]
# i zostawiamy tylko linijki od długości przynajmniej 5 słów
text = [line for line in text if len(line) > 5]

In [ ]:
print(text[:3])

In [ ]:
# lista słów (już bez podziału na linijki)
words = [word for line in text for word in line]

In [ ]:
print(words[:100])

In [ ]:
# zliczamy słowa
word_frequencies = Counter(words)

In [ ]:
# 20 najpopularniejszych słów i liczby wystąpień
word_frequencies.most_common(20)

In [ ]:
# 20 najmniej popularnych słów i liczby wystąpień
word_frequencies.most_common()[:-21:-1]

In [ ]:
# liczba unikalnych słów
len(set(words))

Nie chcemy uczyć zanurzeń wektorowych dla bardzo rzadkich słów (jeśli słowo wystąpiło tylko raz, to czego model może się nauczyć?)
* stosujemy specjalny token UNK (jak unknown), którym zastąpimy rzadkie słowa
* słowa, które wystąpiły w tekście mniej niż 50 razy zastępujemy tokenem UNK

In [ ]:
text = [[word if word_frequencies[word] > 50 else "UNK" for word in line ] for line in text]

In [ ]:
print(text[:3])

In [ ]:
# jeszcze raz lista słów bez podziału na linijki
words = [word for line in text for word in line]

In [ ]:
# tworzymy vocab - listę unikalnych słów, posortowanych alfabetycznie
vocab = sorted(set(words))
# umieszczamy token UNK, tak, żeby miał indeks 0, ponieważ jest specyficzny
vocab.remove('UNK')
vocab = ['UNK'] + vocab
vocab_size = len(vocab)
print('Liczba unikalnych słów: ', vocab_size)
print('Pierwsze (alfabetycznie) słowa:\n', vocab[:10])
print('Ostatnie (alfabetycznie) słowa:\n', vocab[-10:])

In [ ]:
# słownik słowo -> indeks (token)
word_to_idx = {word: i for i, word in enumerate(vocab)}
# słownik indeks -> słowo
idx_to_word = {i: word for i, word in enumerate(vocab)}

In [ ]:
# początek słownika word_to_idx
{k: word_to_idx[k] for k in list(word_to_idx)[:10]}

In [ ]:
# początek słownika idx_to_word
{k: idx_to_word[k] for k in list(idx_to_word)[:10]}

In [ ]:
# tekst zamieniony na tokeny (indeksy)
tokenized_text = [[word_to_idx[word] for word in line] for line in text]

In [ ]:
print(tokenized_text[:3])

## Implementacja algorytmu CBOW

#### Przygotowanie danych wejściowych

* Będziemy uczyć model do predykcji słowa na podstawie jego kontekstu (słów z otoczenia).
* Zastosujemy długość kontekstu 2
* Co oznacza, że przewidujemy słowo na podstawie dwóch poprzednich i dwóch następnych słów

<img src="https://github.com/asztyber/jak_dziala_gpt_lab/blob/main/pic/kontekst.png?raw=true" alt="Kontekst" width="500" height="300">

Dane uczące dla modelu będą wyglądać tak (tylko operujemy na tokenach, a nie na słowach):

<img src="https://github.com/asztyber/jak_dziala_gpt_lab/blob/main/pic/cbow_training.png?raw=true" alt="Dane wejściowe i wyjściowe" width="500" height="300">

#### Funkcja generująca dane do uczenia
* Na podstawie korpusu tekstu po tokenizacji generujemy dane do uczenia
* Zwracana zmienna **data** jest listą krotek, zawierających (lista słów z kontekstu, słowo do przewidzenia (target))
* Uzupełnij wybieranie indeksów dla słów z kontekstu

In [ ]:
def generate_training_data(tokenized_text, context_len):
    data = []
    # dla każdego zdania
    for sentence in tokenized_text:
        sentence_len = len(sentence) # długość zdania
        # uwaga: wewnątrz zadania iterujemy się tylko przez słowa, dla których jest odpowiedni kontekst
        # tj. są przynajmniej dwa słowa przed i dwa słowa za
        for i, word in enumerate(sentence[context_len:-context_len]):
            idx = i + context_len # indeks aktualnego słowa
            context_indices = #TODO wybierz indeksy odpowiadające słowom z kontekstu
            
            data.append(([sentence[ci] for ci in context_indices], word))
    return data

In [ ]:
training_data = generate_training_data(tokenized_text, context_len=2)

In [ ]:
print(tokenized_text[0])

In [ ]:
training_data[:10]

In [ ]:
print(len(training_data))

In [ ]:
assert training_data[:3] == [([0, 2481, 165, 0], 3773), ([2481, 3773, 0, 829], 165), ([3773, 165, 829, 2007], 0)]

In [ ]:
def get_batch(batch_size=8):
    '''
    funkcja zwraca batch danych uczących
    x to tensor o wymiarach (rozmiar batcha x (2 x długość kontekstu)
    y ma wymiar rozmiar_batcha x 1
    x i y zawierają indeksy tokenów i są typu torch.long
    '''
    rand_idx = np.random.randint(0, len(training_data), size=batch_size) # losujemy batch_size losowych indeksów
    x = [training_data[idx][0] for idx in rand_idx]
    y = [training_data[idx][1] for idx in rand_idx]
    x = torch.tensor(x, dtype=torch.long)
    y = torch.tensor(y, dtype=torch.long)
    return x, y

In [ ]:
get_batch(2)

#### Sieć neuronowa
* Zaimplementować model CBOWModel
* *embedding_dim* będzie oznaczać rozmiar reprezentacji wektorowej, którą trenujemy
* Model składa się z:
    * warstwy Embedding o rozmiarze *vocab_size* x *embedding_dim* (*vocab_size* to liczba unikalnych tokenów w naszym tekście) - warstwa musi mieć nazwę embeddings
    * oraz warstwy liniowej (Linear) o wymiarze *embedding_dim* x *vocab_size* (przewidujemy słowo spośród *vocab_size* słów)
* W funkcji forward:
    * Stosujemy warstwę embedding do danych wejściowych
    * Wynikiem jest tensor o wymiarach *batch_size* x *n_words* x *embedding_dim* (*n_words* to liczba słów na wejściu (2 x *context_len*))
    * Uśredniamy embeddingi dla wszystkich słów w obrębie przykładu. Wynikiem powinien być tensor o wymiarze *batch_size* x  *embedding_dim*
    * Stosujemy warstwę liniową

In [ ]:
class CBOWModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CBOWModel, self).__init__() 
        # TODO
        
    def forward(self, input_words): 
        # TODO
        return x

In [ ]:
embedding_dim = 3

In [ ]:
model = CBOWModel(vocab_size, embedding_dim)

In [ ]:
embs = model.embeddings(torch.tensor([[237, 2481, 164, 237], [237, 2481, 164, 237]]))

In [ ]:
assert embs.shape == torch.Size([2, 4, 3])

In [ ]:
out = model(torch.tensor([[237, 2481, 164, 237], [237, 2481, 164, 237]]))

In [ ]:
assert out.shape == torch.Size([2, vocab_size])

### Uczenie modelu

In [ ]:
# To jest wartość funkcji straty dla losowego klasyfikatora
-np.log(1/vocab_size)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
embedding_dim = 128
n_steps = 2000
batch_size = 4096
model = CBOWModel(vocab_size, embedding_dim).to(device)
optimizer = optim.AdamW(model.parameters(), lr=0.003) # AdamW = Adam + regularyzacja L2

#### Zaimplementować pętlę uczenia
* Podpowiedź - laboratorium 4

In [ ]:
for step in range(n_steps):
     # TODO
    if step % 100 == 0:
        print(f"Krok {step}: Loss = {loss.item():.4f}")

#### Analiza uzyskanych wektorów

In [ ]:
def get_word_embedding(word):
    '''
    Funkcja zwraca wektor dla danego słowa
    '''
    word_idx = word_to_idx[word]
    word_tensor = torch.tensor([word_idx], dtype=torch.long).to(device)
    embedding = model.embeddings(word_tensor)
    return embedding.cpu().detach().numpy()

def find_similar(word, top_n=5):
    '''
    Funkcja zwraca top_n słów o najbliższych wektorach dla danego słowa
    '''
    embeddings = model.embeddings.weight.cpu().detach().numpy()
    word_embedding = get_word_embedding(word)
    similarities = cosine_similarity(word_embedding, embeddings)[0]
    sorted_indices = similarities.argsort()[::-1]
    similar_words = [idx_to_word[idx] for idx in sorted_indices[1:top_n+1]]
    return similar_words

In [ ]:
# wektor dla przykładowego słowa
word_embedding = get_word_embedding("network")
word_embedding

In [ ]:
# kształt naszych embeddingów - macierz liczba słów x rozmiar wektora
model.embeddings.weight.shape

##### Zanjdziemy najbliższe słowa dla kilku przykładowych słów

In [ ]:
find_similar("1", 10)

In [ ]:
find_similar("1986", 10)

In [ ]:
find_similar('king', 10)

In [ ]:
find_similar('the', 10)

In [ ]:
find_similar('UNK', 10)

In [ ]:
find_similar('poland', 10)

#### Wizualizacja wybranych słów

In [ ]:
# przykładowe słowa w czterech kategoriach
words_to_vis = ['0', '00', '1', '10', '100', '110', '120', '150', '200', '250', '300', '500', '600', '700',
                '800', '900'] +\
               ['1900', '1920s', '1950s', '1960s', '1970s', '1980s', '1990s', '2000', '10th', '11th', '12th',
                '19th', '20th', '21st'] +\
               ['school', 'teacher', 'students', 'class', 'university', 'college', 'education', 'degree',
                'knowledge'] +\
               ['america', 'argentina', 'australia', 'brazil', 'canada', 'china', 'egypt', 'england',
                'france', 'germany', 'india', 'ireland', 'israel', 'italy', 'japan', 'mexico', 'norway',
                'palestine', 'poland', 'portugal', 'russia', 'somalia', 'spain', 'vietnam']

In [ ]:
# wyznaczamy wektory
embs = np.array([get_word_embedding(word)[0] for word in words_to_vis])
embs.shape

In [ ]:
# Za pomocą algorytmu T-SNE znajdujemy znaurzenia wektorów w przestrzeń 2d
tsne = TSNE(n_components=2, random_state=42)
embs_2d = tsne.fit_transform(embs)

In [ ]:
embs_2d.shape

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(embs_2d[:, 0], embs_2d[:, 1], alpha=0.6)
for i in range(len(words_to_vis)):
    plt.annotate(words_to_vis[i], (embs_2d[:, 0][i], embs_2d[:, 1][i]),
                 textcoords="offset points", xytext=(5, 5), ha='center', fontsize=7, color='black')

#### Wnioski
1. Skomentuj powyższy wykres
2. Znajdź dwa słowa, dla których słowa o najbliższych wektorach wyglądają rozsądnie
3. Znajdź dwa słowa, dla których słowa o najbliższych wektorach wyglądają zaskakująco lub bez sensu
4. Jaka może być przyczyna (3)?
5. [dla chętnych] Lepsze parametry uczenia (rozmiar wektorów, współczynnik uczenia, optymalizator, rozmiar batcha, ...)